<a href="https://colab.research.google.com/github/GrzegorzMeller/AlgorithmsForMassiveData/blob/master/OCULAR_DISEASE_AUGMENTATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/amd/')

Drive already mounted at /amd/; to attempt to forcibly remount, call drive.mount("/amd/", force_remount=True).


In [0]:
!cp /amd/My\ Drive/ODIR-5K.zip /content/
!unzip ODIR-5K.zip

In [8]:
def show_class_distribution():
  N = 0
  D=0
  G=0
  C=0
  A=0
  H=0
  M=0
  O = 0
  X = 0
  for element in glob.glob("ODIR-5K/Training Images/*.jpg"):
    img_name = element.split("/")[-1]
    img_label = img_name.split("-")
    if img_label[-1] == "N.jpg":
      N += 1
    elif img_label[-1] == "D.jpg":
      D +=1
    elif img_label[-1] == "G.jpg":
      G +=1
    elif img_label[-1] == "C.jpg":
      C +=1
    elif img_label[-1] == "A.jpg":
      A +=1
    elif img_label[-1] == "H.jpg":
      H +=1
    elif img_label[-1] == "M.jpg":
      M +=1
    elif img_label[-1] == "O.jpg":
      O +=1
    elif "&" in img_label[-1]:
      X +=1  
  print(N, D, G, C, A, H, M, O, X)
show_class_distribution()

2476 1213 193 233 205 92 212 657 483


In [0]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import random
import glob
from PIL import Image, ImageEnhance

def cv2_clipped_zoom(img, zoom_factor):
    height, width = img.shape[:2] # It's also the final desired shape
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)
    y1, x1 = max(0, new_height - height) // 2, max(0, new_width - width) // 2
    y2, x2 = y1 + height, x1 + width
    bbox = np.array([y1,x1,y2,x2])
    # Map back to original image coordinates
    bbox = (bbox / zoom_factor).astype(np.int)
    y1, x1, y2, x2 = bbox
    cropped_img = img[y1:y2, x1:x2]

    # Handle padding when downscaling
    resize_height, resize_width = min(new_height, height), min(new_width, width)
    pad_height1, pad_width1 = (height - resize_height) // 2, (width - resize_width) //2
    pad_height2, pad_width2 = (height - resize_height) - pad_height1, (width - resize_width) - pad_width1
    pad_spec = [(pad_height1, pad_height2), (pad_width1, pad_width2)] + [(0,0)] * (img.ndim - 2)

    result = cv.resize(cropped_img, (resize_width, resize_height))
    result = np.pad(result, pad_spec, mode='constant')
    assert result.shape[0] == height and result.shape[1] == width
    return result

def aumgent_9(impath):
  img = cv.imread(impath,0)
  im1 = cv.flip(img, 0) #Vertical Flip
  im2 = cv.flip(img, 1) #Horizontal Flip
  im3 = cv.flip(img, -1) #Flip Both
  im4 = cv2_clipped_zoom(img, random.uniform(0.9,1.3))
  im5 = cv2_clipped_zoom(im1, random.uniform(0.9,1.3))
  im6 = cv2_clipped_zoom(im2, random.uniform(0.9,1.3))
  file_name = impath.split('/')
  new_path = file_name[-1].split('-') 
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"VF-"+new_path[1],im1)
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"HF-"+new_path[1],im2)
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"BF-"+new_path[1],im3)
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"ZM-"+new_path[1],im4)
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"VFZM-"+new_path[1],im5)
  #image random rotating and brightening using PIL
  img_org = Image.open(impath)
  enhancer = ImageEnhance.Brightness(img_org)
  im7 = enhancer.enhance(random.uniform(1.0,1.3))
  im8 = img_org.rotate(random.uniform(0,10))
  img_ver = Image.open("ODIR-5K/Training Images/"+new_path[0]+"VF-"+new_path[1])
  im9 = img_ver.rotate(random.uniform(0,10))
  im7.save("ODIR-5K/Training Images/"+new_path[0]+"RB-"+new_path[1])
  im8.save("ODIR-5K/Training Images/"+new_path[0]+"RR-"+new_path[1])
  im9.save("ODIR-5K/Training Images/"+new_path[0]+"VFRR-"+new_path[1])


#aumgent_1 will be applied to class D and O, since they don't need to be aumgmented so much in order to balance the dataset
def aumgent_1(impath):
  img = cv.imread(impath,0)
  im1 = cv.flip(img, 0) #Vertical Flip
  file_name = impath.split('/')
  new_path = file_name[-1].split('-') 
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"VF-"+new_path[1],im1)



In [0]:
labels = ["G.jpg", "C.jpg", "A.jpg", "H.jpg", "M.jpg"]
labels2 = ["D.jpg", "O.jpg"]

for element in glob.glob("ODIR-5K/Training Images/*.jpg"):
  file_name = element.split('/')
  new_path = file_name[-1].split('-') 
  for l in labels:
    if new_path[-1] == l:
      aumgent_9(element)
  for l in labels2:
    if new_path[-1] == l:
      aumgent_1(element)

In [12]:
show_class_distribution()

2476 2426 1737 2097 1845 828 1908 1314 483


In [0]:
!zip -r ODIR-5K.zip ODIR-5K/

In [0]:
!cp -r ODIR-5K.zip /amd/My\ Drive

In [0]:
!rm -rf ODIR-5K/